In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

In [2]:
train_dataset=datasets.FashionMNIST(root='../dataset',train=True,transform=transforms.ToTensor(),download=True)
test_dataset=datasets.FashionMNIST(root='../dataset',train=False,transform=transforms.ToTensor())

100%|██████████| 26421880/26421880 [00:02<00:00, 9765559.64it/s] 


Extracting ../dataset/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../dataset/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 169871.74it/s]


Extracting ../dataset/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../dataset/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3173463.41it/s]


Extracting ../dataset/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../dataset/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 18205967.11it/s]

Extracting ../dataset/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../dataset/FashionMNIST/raw



In [3]:
batch_size=64
num_epochs=50
learning_rate=1e-3
device=torch.cuda.is_available()

In [4]:
train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

In [5]:
class neuralNetwork(nn.Module):
  def __init__(self,in_dim,n_hidden_1,n_hidden_2,out_dim):
    super().__init__()
    self.layer1=nn.Sequential(
        nn.Linear(in_dim,n_hidden_1),
        nn.ReLU(True)
    )
    self.layer2=nn.Sequential(
        nn.Linear(n_hidden_1,n_hidden_2),
        nn.ReLU(True)
    )
    self.layer3=nn.Sequential(
        nn.Linear(n_hidden_2,out_dim),
        nn.ReLU(True)
        )
  def forward(self,x):
    x=self.layer1(x)
    x=self.layer2(x)
    x=self.layer3(x)
    return x

In [6]:
model=neuralNetwork(28 * 28,300,100,10)

In [7]:
if device:
  model=model.cuda()

In [8]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [9]:
for epoch in range(num_epochs):
  print('*' * 10)
  print(f'epoch{epoch+1}')
  running_loss=0.0
  running_acc=0.0
  for i,data in enumerate(train_loader,1):
    img,label=data
    img=img.view(img.size(0),-1)
    if device:
      img=img.cuda()
      label=label.cuda()
    out=model(img)
    loss=criterion(out,label)
    running_loss+=loss.item()
    _,pred=torch.max(out,1)
    running_acc+=(pred==label).float().mean()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i % 300 ==0:
      print(f'epoch: {epoch+1}/{num_epochs},loss: {running_loss/i:.6f},acc: {running_acc/i:.6f}')
  print(f'Finish:{epoch+1} epoch,loss: {running_loss/i:.6f},acc: {running_acc/i:.6f}')



**********
epoch1
epoch: 1/50,loss: 1.022195,acc: 0.655052
epoch: 1/50,loss: 0.925603,acc: 0.676927
epoch: 1/50,loss: 0.877534,acc: 0.689375
Finish:1 epoch,loss: 0.874156,acc: 0.690099
**********
epoch2
epoch: 2/50,loss: 0.764056,acc: 0.717344
epoch: 2/50,loss: 0.754316,acc: 0.720573
epoch: 2/50,loss: 0.745170,acc: 0.723785
Finish:2 epoch,loss: 0.745429,acc: 0.723464
**********
epoch3
epoch: 3/50,loss: 0.597815,acc: 0.801042
epoch: 3/50,loss: 0.580986,acc: 0.803724
epoch: 3/50,loss: 0.573987,acc: 0.804948
Finish:3 epoch,loss: 0.572239,acc: 0.805504
**********
epoch4
epoch: 4/50,loss: 0.530607,acc: 0.816042
epoch: 4/50,loss: 0.525092,acc: 0.818906
epoch: 4/50,loss: 0.526865,acc: 0.817830
Finish:4 epoch,loss: 0.526986,acc: 0.817747
**********
epoch5
epoch: 5/50,loss: 0.507967,acc: 0.822188
epoch: 5/50,loss: 0.512970,acc: 0.819818
epoch: 5/50,loss: 0.508572,acc: 0.821632
Finish:5 epoch,loss: 0.507721,acc: 0.821978
**********
epoch6
epoch: 6/50,loss: 0.483182,acc: 0.831823
epoch: 6/50,loss

In [10]:
model.eval()
eval_loss=0.0
eval_acc=0.0
for data in train_loader:
  img,label=data
  img=img.view(img.size(0),-1)
  if device:
    img=img.cuda()
    label=label.cuda()

  with torch.no_grad():
    out=model(img)
    loss=criterion(out,label)
    eval_loss+=loss.item()
    _,pred=torch.max(out,1)
    eval_acc+=(pred==label).float().mean()
print(f'Test loss: {eval_loss/len(train_loader)},acc: {eval_acc/len(train_loader)}')

Test loss: 0.3154634066871298,acc: 0.8729677796363831


In [11]:
torch.save(model.state_dict(),'./neuralNetwork.pth')